# Local LLM LangChain v1 Agent

This notebook demonstrates how to wire a LangChain v1 agent to a locally hosted LLM (default: [Ollama](https://ollama.ai)).


## Prerequisites

- Start your local model server (e.g., run `ollama serve` after pulling a model such as `ollama pull llama3`).
- Install the project dependencies with `pip install -r requirements.txt` inside a virtual environment.
- Create a `.env` file (see `.env.example`) with `MODEL_NAME` and `OLLAMA_BASE_URL`.


In [ ]:
# Optional: install dependencies directly from the notebook (uncomment if needed)
# %pip install -r ../requirements.txt


In [ ]:
import os
import datetime as dt
import platform
from typing import Any

from dotenv import load_dotenv
from langchain.agents import AgentExecutor, create_react_agent
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.tools import tool


In [ ]:
# Load env vars and define sane defaults for local development
load_dotenv()
MODEL_NAME = os.getenv("MODEL_NAME", "llama3")
OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL", "http://localhost:11434")
TEMPERATURE = float(os.getenv("MODEL_TEMPERATURE", "0.2"))
print(f"Using model={MODEL_NAME} @ {OLLAMA_BASE_URL}")


In [ ]:
# Instantiate the local LLM client (Ollama by default)
llm = ChatOllama(
    base_url=OLLAMA_BASE_URL,
    model=MODEL_NAME,
    temperature=TEMPERATURE,
)


In [ ]:
@tool
def system_context(_: str = "") -> str:
    """Return contextual information about the workstation (OS, time)."""
    now = dt.datetime.now().isoformat(timespec="seconds")
    return f"System time: {now}\nPlatform: {platform.platform()}"


@tool
def python_math(expression: str) -> str:
    """Safely evaluate simple math expressions using Python syntax."""
    allowed_globals: dict[str, Any] = {"__builtins__": {}}
    try:
        result = eval(expression, allowed_globals, {})
    except Exception as exc:  # noqa: BLE001 - provide feedback to agent
        return f"error: {exc}"
    return str(result)


tools = [system_context, python_math]
for t in tools:
    print(f"Loaded tool: {t.name}")


In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful local-only AI agent. Use the available tools when helpful before responding."),
    ("placeholder", "{chat_history}"),
    ("user", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


## Run the agent

Use `agent_executor.invoke({"input": "<prompt>"})` to get a structured response. The `output` key carries the final answer.


In [ ]:
query = "What's the current system context and what is (42 * 17) - 5?"
result = agent_executor.invoke({"input": query})
result["output"]
